In [1]:
import pandas as pd
import numpy as np
import graph_tool.all as gt

In [2]:
from os import path
from json import load
from collections import Counter, defaultdict

# Import triples

In [3]:
clustered_triples = True

In [4]:
if clustered_triples:
    file_names = ['triples_wizardLM_filtering.json']
    folder_path = path.join('..', 'outputs', 'genSRL', 'clustered_t80')
else:
    file_names = ['triples_wizardLM_filtering_setA.json', 'triples_wizardLM_filtering_setB.json', 'triples_wizardLM_filtering_setC.json', 'triples_wizardLM_filtering_setD.json', 'triples_wizardLM_filtering_setE.json']
    folder_path = path.join('..', 'outputs', 'genSRL')

In [5]:
triple_data = dict()
for file_name in file_names:
    file_path = path.join(folder_path, file_name)
    with open(file_path) as fp:
        triple_data.update(load(fp))
print("COMPANIES:", len(triple_data.keys()), '\n')
print('\n'.join([ f'{idk + 1}) {companyName}'for idk, companyName in enumerate(sorted(triple_data.keys()))]))

COMPANIES: 128 

1) 3M Corporation
2) 3i Group plc
3) Activision Blizzard Inc
4) Adecco Group AG
5) Adidas AG
6) Air Canada
7) Air Liquide SA
8) Airbus SE
9) Alcon Inc
10) Alibaba Group Holding Limited
11) Alphabet Inc
12) Aluminum Corporation of China Limited
13) Amazoncom Inc
14) American Electric Power Company Inc
15) Amplifon
16) Apple Inc
17) ArcelorMittal SA
18) Assicurazioni Generali SpA
19) AstraZeneca PLC
20) AstraZeneca plc
21) BPER Banca SpA
22) Baidu Inc
23) Banco Santander SA
24) Bank of America Corp BofA
25) Bayer AG
26) British American Tobacco PLC
27) British American Tobacco plc
28) British Land Co PLC The
29) Broadcom Inc
30) Builders FirstSource Inc
31) CF Industries Holdings Inc
32) Campbell Soup Company
33) Canadian Pacific Railway Limited
34) Canon Inc
35) CarMax Inc
36) China Evergrande Group
37) China Petroleum Chemical Corporation
38) Cisco Systems Inc
39) Coca Cola
40) Commonwealth Bank of Australia
41) Croda International plc
42) Daikin Industries Ltd
43) Del

In [8]:
raw_data = []
for companyName, triples in triple_data.items():
    for triple in triples:
        raw_data.append({
            'company': companyName,
            'category': triple['esg_category'],
            'predicate': triple['predicate'],
            'object': triple['object'],
            'original': triple['properties']['original_sentence']
        })
df = pd.DataFrame(raw_data)
df['action'] = df['category'].str.upper() + ': ' + df['predicate'].str.lower() #+ ' ' + df['object'].str.lower() 
display(df)

,company,category,predicate,object,original,action
0,3M Corporation,Access to Basic Services,Availability of education,"In areas including medical, dental, orthodonti...",Education is available in areas including medi...,ACCESS TO BASIC SERVICES: availability of educ...
1,3M Corporation,Access to Basic Services,Challenges related to,Land ownership and engagement in forestry,African Americans in the rural south have long...,ACCESS TO BASIC SERVICES: challenges related to
2,3M Corporation,Access to Healthcare,Availability of education,A broad range of platforms to help assure acce...,As digitization plays an increasingly importan...,ACCESS TO HEALTHCARE: availability of education
3,3M Corporation,Accessibility,Approaching,Accessibility with the understanding that our ...,We are approaching accessibility with the unde...,ACCESSIBILITY: approaching
4,3M Corporation,Accessibility,Design for,Easily printable website pages,The website pages are designed to be easily pr...,ACCESSIBILITY: design for
...,...,...,...,...,...,...
49792,adidas AG,Supply Chain,Description of,"The greatest labor and environmental, health, ...",Description of the greatest (1) labor and (2) ...,SUPPLY CHAIN: description of
49793,adidas AG,Supply Chain,Number of,Tier 1 suppliers and suppliers beyond Tier 1,Number of (1) Tier 1 suppliers and (2) supplie...,SUPPLY CHAIN: number of
49794,adidas AG,Supply Chain,Working with,520 independent supplier facilities (Tier 1),At the end of 2020 adidas worked with 520 inde...,SUPPLY CHAIN: working with
49795,adidas AG,Product Sustainability,Standardization of,SASB's standards for sustainability disclosure,1. The Sustainability Accounting Standards Boa...,PRODUCT SUSTAINABILITY: standardization of


In [11]:
df['original'].sort_values(ascending=False).reset_index(drop=True).to_frame()

,original
0,������������������������������������� ��������...
1,�To take a truly holistic and proactive perspe...
2,�Identified further areas/assets where a green...
3,� ��������������������������������������������...
4,� In addition to the tried and tested pandemic...
...,...
49792,"""With the green loan from UniCredit HypoVerein..."
49793,"""To effect change we have to redouble our effo..."
49794,"""GHG Emissions Actual and Projected (MMT CO2e)..."
49795,"""Fresh water"" includes water sourced from fres..."


In [ ]:
for _,item in df.sample(5).iterrows():
    display(item)

# Get the company info

In [ ]:
company_info = pd.read_excel(path.join('..', 'data', 'companySectors_auto.xlsx'), index_col = 0)
company_info = company_info.loc[df['company'].unique(), :] #[item.lower() for item in items]
company_info['tmp'] = company_info.index.str.lower()
company_info = company_info.drop_duplicates(subset = ['tmp']).drop(columns = ['tmp'])
display(company_info)

# Group by sector
companyBySector = company_info.copy()
companyBySector['Companies'] = companyBySector.index
companyBySector = companyBySector[['Companies','Sector']].groupby('Sector').agg(Counter)
companyBySector.insert(loc = 0, column = 'Count', value = companyBySector['Companies'].map(Counter.total))
companyBySector['Companies'] = companyBySector['Companies'].map(lambda counter: ', '.join(sorted(counter.keys())))
companyBySector = companyBySector.sort_values(by = 'Count', ascending = False)
display(companyBySector)

In [ ]:
with pd.ExcelWriter(path.join('..', 'data', 'selectedCompanies.xlsx'), ) as writer:
    company_info[['yahooName','Sector', 'Industry', 'Ticker']].to_excel(writer, sheet_name='Companies')
    companyBySector.to_excel(writer, sheet_name = 'Sectors')

## Entropy

In [ ]:
# Merge the main dataframe with the company info
merged_df = df.merge(company_info, left_on='company', right_index=True)

# Compute the number of actions per company
numCompanyActions = df[['company', 'action']].groupby('company').count().rename(columns = {'action':'actions'})
numActionsBySector = merged_df[['Sector', 'action']].groupby('Sector').count().rename(columns = {'action':'actions'})

node_counters = dict()
sort_dict = lambda counter: dict(sorted(counter.items(), key=lambda dict_item: (dict_item[1], dict_item[0]), reverse = True))
for col in ['category', 'action']:
    
    print('-' * 50, col, '-' * 50, '\n')
    
    # Count the values
    counter_df = merged_df[[col, 'company', 'Sector']].groupby(by = col).agg(Counter)
    counter_df['company'] = counter_df['company'].map(sort_dict)
    counter_df['Sector'] = counter_df['Sector'].map(sort_dict)
    
    # NEW COLUMN 0: item counter
    counter_df.insert(loc = 0, column = 'count', value = (counter_df['company'].map(Counter.total) / merged_df.index.size).round(3))
    
    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # --------------------------------------------------------------- COMPANIES ------------------------------------------------------------------------------------------------------
    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
    # NEW COLUMN 1A: [COMPANY] company cardinality
    counter_df['company cardinality'] = counter_df['company'].map(lambda companyDict: np.round(len(companyDict.values()) / company_info.index.size, 2)) 
    
    # NEW COLUMN 1B: [COMPANY] action per company (%)
    counter_df['action per company (%)'] = counter_df['company'].map(
        lambda dict_counter: dict(sorted([(company, round(action_counter / numCompanyActions.loc[company, 'actions'], 2)) for company, action_counter in dict_counter.items()], 
                                    key = lambda x: (x[1], x[0]), reverse = True)))
    
    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # --------------------------------------------------------------- SECTOR ---------------------------------------------------------------------------------------------------------
    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    # NEW COLUMN 2A: Company sector coverage (%)
    counter_df['company sector coverage (%)'] = counter_df['company'].map(lambda counter: Counter([company_info.loc[company, 'Sector'] for company in counter.keys()]))
    counter_df['company sector coverage (%)'] = counter_df['company sector coverage (%)'].map(
        lambda counter: dict(sorted([(sector, round(counter / companyBySector.loc[sector, 'Count'], 4)) for sector, counter in counter.items()], 
                                    key = lambda x: (x[1], companyBySector.loc[x[0], 'Count'], x[0]), reverse=True)))
    
    # NEW COLUMN 2B: company sector cardinality
    counter_df['company sector cardinality'] = counter_df['company sector coverage (%)'].map(lambda counter: np.round(len(counter.keys()) / companyBySector.index.size, 2))
    
    # NEW COLUMN 2C: Action per sector (%)
    counter_df['action per sector (%)'] = counter_df['Sector'].map(
        lambda counter: dict(sorted([(sector, round(action_counter / numActionsBySector.loc[sector, 'actions'], 4)) for sector, action_counter in counter.items()], 
                                    key = lambda x: (x[1], companyBySector.loc[x[0], 'Count'], x[0]), reverse = True)))
    #counter_df['action per sector (%)'] = counter_df['action per sector (%)'].map(lambda counter: ', '.join([item + f' ({np.round(count * 100, 1)} %)' for item, count in counter.items()]))

    # Sort dataframe according to the counter
    counter_df = counter_df.sort_values(by = ['count', 'company cardinality'], ascending = False)
    counter_df = counter_df.rename(columns = {'company': 'action per company', 'Sector': 'action per sector'})
    counter_df = counter_df.reindex(
        columns = ['count', 'company cardinality', 'action per company', 'action per company (%)', 'company sector cardinality',  'company sector coverage (%)', 'action per sector', 'action per sector (%)'])
    
    # Save the dataframe
    node_counters[col] = counter_df
    display(counter_df)

    # Grouped by sector
    sectorCounter_df = merged_df[['Sector', 'company', col]].groupby(['Sector', col]).agg(Counter)
    sectorCounter_df['count'] = sectorCounter_df['company'].map(Counter.total)
    
    if col == 'action':
        sectorCounter_df.index = pd.MultiIndex.from_tuples(
            tuples = [(sector, action.split(':')[0].strip().capitalize(), action) for sector, action in sectorCounter_df.index], 
            names = ['Sector', 'category', 'action'])
    
    # Visual improvements
    sectorCounter_df['company'] = sectorCounter_df['company'].map(lambda counter: dict(sorted(counter.items(), key=lambda x: x[1], reverse=True)))
    sectorCounter_df['idk'] = sectorCounter_df.index.get_level_values(0)
    sectorCounter_df = sectorCounter_df[['idk', 'count', 'company']].rename(columns = {'company':'companies'}).sort_values(by = ['idk', 'count'], ascending = [True, False]).drop(columns = ['idk'])

    node_counters[col + 'bySector'] = sectorCounter_df
    
    display(sectorCounter_df)


## Entropy

In [ ]:
from scipy.stats import entropy, pearsonr
def compute_entropy(labels, base = None):
  _, counts  = np.unique(labels, return_counts = True)
  probs = counts / counts.sum()  
  
  entropy_value = entropy(probs, base = base)
  print("Entropy:", entropy_value)
  print("Entropy:",  entropy(labels, base = base))
  return entropy_value

#### Category - Predicate

In [ ]:
action_entropy_df = merged_df[['category', 'predicate', 'company']].groupby('category').agg(list)
action_entropy_df['entropy'] = action_entropy_df['predicate'].map(lambda predicates: [pred.lower() for pred in predicates]).apply(lambda predicates: compute_entropy(predicates))

display()

action_entropy_df['predicate'] = action_entropy_df['predicate'].map(Counter)\
    .map(lambda counter: {item: count / np.sum(list(counter.values())) for item, count in counter.items()})\
        .map(lambda counter: dict(sorted(counter.items(), key=lambda item: item[1], reverse=True)))

action_entropy_df['num_predicates'] = action_entropy_df['predicate'].map(lambda counter: len(counter.keys()))
action_entropy_df['company'] = action_entropy_df['company'].map(lambda companies: len(set(companies)) /  company_info.index.size)

action_entropy_df['predicate'] = action_entropy_df['predicate'].map(lambda counter: ', '.join([item + f' ({np.round(count * 100, 1)} %)' for item, count in counter.items()]))

action_entropy_df['normalized_entropy'] = (action_entropy_df['entropy'] - action_entropy_df['entropy'].min()) / (action_entropy_df['entropy'].max() - action_entropy_df['entropy'].min())


entropyCompanies_corr = pearsonr(action_entropy_df['company'], action_entropy_df['entropy'])
print("[CORRELATION]: entropy-companies", round(entropyCompanies_corr[0], 2))

entropyCompanies_corr = pearsonr(action_entropy_df['company'], action_entropy_df['normalized_entropy'])
print("[CORRELATION]: normalised entropy-companies", round(entropyCompanies_corr[0], 2))

action_entropy_df = action_entropy_df[['entropy','company','num_predicates' ,'predicate']].sort_values(['entropy', 'company'], ascending=False).round(2)
display(action_entropy_df)

node_counters['action_entropy'] = action_entropy_df

#### Category - Predicate by sector

In [ ]:
action_entropybySector_df = merged_df[['Sector', 'category', 'predicate']]
action_entropybySector_df = action_entropybySector_df.groupby(['Sector', 'category']).agg(list) 
action_entropybySector_df['entropy'] = action_entropybySector_df['predicate'].map(lambda predicates: [pred.lower() for pred in predicates]).apply(lambda predicates: compute_entropy(predicates))

action_entropybySector_df = action_entropybySector_df.reset_index()[['Sector', 'entropy']].groupby('Sector').agg({'entropy': ['median','mean','std']})
action_entropybySector_df.columns = [col[1] + '_' + col[0] for col in action_entropybySector_df.columns]

action_entropybySector_df = action_entropybySector_df.sort_values(by = ['median_entropy', 'mean_entropy'], ascending=False) # type: ignore

node_counters['action_sectorEntropy']  = action_entropybySector_df
display(action_entropybySector_df)

## Save findings

In [ ]:
folder_path = path.join('..', 'outputs', 'graph_analyses')

for method, counter_df in node_counters.copy().items():
    if 'by' not in method:
        file_path = path.join(folder_path, 'sector_stats_' + method + ('_clustered' if clustered_triples else '') + '.json')
        counter_df.to_json(file_path, orient = 'index', indent = 4) # orient = 'index', 
        
        print('METHOD', method)

In [ ]:
stringy_func = lambda items: ', '.join([ f'{name} ({count})' for name, count in items.items()])
with pd.ExcelWriter(path.join(folder_path, 'sector_stats' + ('_clustered' if clustered_triples else '') + '.xlsx')) as writer:
    
    # Aggregated stats
    for sheet_name, counter_df in node_counters.items():
        
        to_save = counter_df.copy()
        
        # Visual improvements: strinfy dictionarioes
        cols_to_stringify = ['action per company', 'action per sector', 'action per company (%)', 'action per sector (%)', 'company sector coverage (%)']
        for col in cols_to_stringify:
            if col in counter_df.columns:
                if '%' in col:
                    to_save[col] = to_save[col].map(lambda items: ', '.join([ f'{name} ({np.round(count*100, 1)}%)' for name, count in items.items()]))
                else:
                    to_save[col] = to_save[col].map(stringy_func)
        
        # Visual improvements: uppercase column names
        to_save.columns = [col.upper() for col in to_save.columns]
        
        # Save to excel
        to_save.to_excel(writer, sheet_name = sheet_name)
        
    # Save info
    companyBySector.to_excel(writer, sheet_name = 'info')